In [47]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import os
import platform
import time
import json
from PyPDF2 import PdfFileReader
from openpyxl import load_workbook
import io

import numpy as np
import pandas as pd
import re
import array
from time import sleep
from os.path import exists
from google_play_scraper import app

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [48]:
# click privacy policy button, pass in webdriver and data safety urls with privacy policy button on page
def press_button(driver, data_url):
    driver.implicitly_wait(1)
    driver.get(data_url)
    print(data_url)
    # press privacy policy button by LINK_TEXT from html/web inspection page
    
    try:
        button = driver.find_element(By.LINK_TEXT, "privacy policy")
        button.click()
    
    except NoSuchElementException:
        pass
        current = "N/A"
        print(current)
        return current
    
    
    # close unused tabs, url for privacy policy is child window
#     p = driver.window_handles[0]
#     c = driver.window_handles[1]
#     driver.switch_to.window(c)
    
#     WebDriverWait(driver, 2)
    
#     current = driver.current_url
#     print(current)
    comp = driver.current_url
    tabs_open = len(driver.window_handles)

    if tabs_open > 1:        
        p = driver.window_handles[0]
        c = driver.window_handles[1]
        driver.switch_to.window(c)

        WebDriverWait(driver, 4)

        current = driver.current_url
        print(current)
        # close child window and switch to original/parent window
        driver.close()
    
        driver.switch_to.window(p)
    else:
        WebDriverWait(driver, 2)
        current = driver.current_url
        print(current)

    sleep(1)

    # return privacy policy url for df
    return current

In [49]:
# scroll search page to get all returned app url's and id's
def scroll(driver, timeout):
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_pause_time = timeout
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
            
        last_height = new_height

In [50]:
# get urls from search
# def get_urls(starts_with):
#     http = httplib2.Http()
#     chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument("--incognito")
#     if platform.system() == 'Windows':
#         driver = webdriver.Chrome(executable_path='./chromedriver.exe')
#     else:
#         if platform.system() == 'Darwin':
#             driver = webdriver.Chrome()
#     # wait for page to load
#     driver.implicitly_wait(30)
#     # provide url(s) for driver
#     driver.get("https://play.google.com/store/search?q=" + starts_with  + "&c=apps")
#     # scroll to get all apps
#     scroll(driver, 2)

#     # read and obtain all urls
#     if platform.system() == 'Windows':
#         soup_a = BeautifulSoup(driver.page_source, features='html.parser')
#     else:
#         if platform.system() == 'Darwin':
#             soup_a = BeautifulSoup(driver.page_source, 'lxml')
    
#     driver.close()

#     # copy all urls
#     urls = []
#     for link in soup_a.find_all('a'):
#         urls.append(link.get('href'))
#     return urls

In [51]:
# alphabet = "abcdefghijklmnopqrstuvwxyz"
# categories = ["Daydream", "Beauty", "Auto & Vehicles", "Business", "Comics", "Books & Reference", "Dating", "Education", "Communication", "Food & Drink", "Entertainment", "Health & Fitness", "Events", "Finance", "Libraries & Demo", "House & Home", "Lifestyle", "Medical", "Maps & Navigation", "News & Magazines", "Audio", "Personalization", "Photography", "Parenting", "Shopping", "Productivity", "Tools", "Social", "Sports", "Video Players & Editors", "Travel & Local", "Weather",
#               "Adventure", "Arcade", "Action", "Board", "Casino", "Casual", "Card", "Music", "Puzzle", "Educational", "Role Playing", "Racing", "Sports", "Strategy", "Simulation", "Brain Games", "Trivia", "Word", "Education", "Creativity", "Video", "TV", "Weather", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
# categories = ['word']
# urls = []
# alphabet = 'ab'

# loop through search categories and add to urls array
# for char_index in range(len(categories)):
#     print(categories[char_index])
#     urls += get_urls(categories[char_index])
#     print(urls[char_index])
# print(len(urls))
# for i in range(len(urls)):
#     print(urls[i])


In [52]:
def get_data_safety_urls(app_ids):
    
    base_url = "https://play.google.com/store/apps/datasafety?id="
    data_urls = []
    
    data_urls = [base_url + s for s in app_ids]
    
    return data_urls
            

In [53]:
def create_app_data_dataframe(unique_apps):
    unique_app_ids = []
    invalid_app_ids=[]
    app_names = []
    app_categories=[]
    app_downloads=[]
    app_score=[]
    app_ratings=[]
    app_developer_name=[]
    app_developer_address=[]
    app_version_number=[]
    app_privacy_policy_url=[]
    app_privacy_policy_date=[]
 

    for app_id in unique_apps:
        try:
            app_data = app(app_id)
        except:
            invalid_app_ids.append(app_id)
        else:
            unique_app_ids.append(app_data['appId'])    
            app_names.append(app_data['title'])
            app_categories.append(app_data['genre'])
            app_downloads.append(app_data['realInstalls'])
            app_score.append(app_data['score'])
            app_ratings.append(app_data['ratings'])
            app_developer_name.append(app_data['developer'])
            app_developer_address.append(app_data['developerAddress'])
            app_version_number.append(app_data['version'])
            app_privacy_policy_url.append(app_data['privacyPolicy'])
            app_privacy_policy_date.append(app_data['released'])

    if(exists('./invalid_app_ids_list.xlsx')):
        invalid_app_data=pd.read_excel('./invalid_app_ids_list.xlsx')
        invalid_app_df = pd.DataFrame({"Invalid App IDs": invalid_app_data['Invalid App IDs'].to_list()+invalid_app_ids})
        invalid_app_df.to_excel (r'./invalid_app_ids_list.xlsx', index = False, header=True)
    else:
        invalid_app_df = pd.DataFrame({"Invalid App IDs": invalid_app_ids})
        invalid_app_df.to_excel (r'./invalid_app_ids_list.xlsx', index = False, header=True)         

    if(exists('./crawled_app_metadata.xlsx')):
        app_metadata = pd.read_excel('./crawled_app_metadata.xlsx')            
        app_metadata_obj={
            "Application Name": app_metadata['Application Name'].to_list()+app_names,
            "Application ID": app_metadata['Application ID'].to_list()+unique_app_ids,
            "Category": app_metadata['Category'].to_list()+app_categories,
            "Downloads": app_metadata['Downloads'].to_list()+app_downloads,
            "Score": app_metadata['Score'].to_list()+app_score,
            "Ratings": app_metadata['Ratings'].to_list()+app_ratings,
            "Developer Name": app_metadata['Developer Name'].to_list()+app_developer_name,
            "Developer Address": app_metadata['Developer Address'].to_list()+app_developer_address,
            "Version": app_metadata['Version'].to_list()+app_version_number,
            "Privacy Policy URL": app_metadata['Privacy Policy URL'].to_list()+app_privacy_policy_url,
            "Privacy Policy Date": app_metadata['Privacy Policy Date'].to_list()+app_privacy_policy_date
        }
        app_metadata_df = pd.DataFrame(app_metadata_obj)
        app_metadata_df.to_excel (r'./crawled_app_metadata.xlsx', index = False, header=True)
    else:
        app_metadata_obj={
            "Application Name": app_names,
            "Application ID": unique_app_ids,
            "Category": app_categories,
            "Downloads": app_downloads,
            "Score": app_score,
            "Ratings": app_ratings,
            "Developer Name": app_developer_name,
            "Developer Address": app_developer_address,
            "Version": app_version_number,
            "Privacy Policy URL": app_privacy_policy_url,
            "Privacy Policy Date": app_privacy_policy_date
        }
        app_metadata_df = pd.DataFrame(app_metadata_obj)
        app_metadata_df.to_excel (r'./crawled_app_metadata.xlsx', index = False, header=True)

In [54]:
def get_similar_app_ids_from_search(app_ids, start_index, end_index, data):
    print('Starting Play Store Crawl for similar apps......')
    similar_or_other_app_ids = []
    app_ids_searched=[]
    unique_app_list = pd.read_excel('./unique_app_ids_list.xlsx')
    for index,app_id in enumerate(app_ids[start_index:end_index+1]):
        url = data['play_store_search_url']+app_id+'&c=apps' 
        http = httplib2.Http()
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--incognito")
        if platform.system() == 'Windows':
            driver = webdriver.Chrome(executable_path='./chromedriver.exe')
        else:
            if platform.system() == 'Darwin':
                driver = webdriver.Chrome()
        
        driver.implicitly_wait(30)
        # provide url(s) for driver
        driver.get(url)
        # scroll to get all apps
        scroll(driver, 2)

        # read and obtain all urls
        if platform.system() == 'Windows':
            soup_a = BeautifulSoup(driver.page_source, features='html.parser')
        else:
            if platform.system() == 'Darwin':
                soup_a = BeautifulSoup(driver.page_source, 'lxml')    
        driver.close()
        # copy all urls
        similar_app_urls = []
        for link in soup_a.find_all('a'):
            similar_app_urls.append(link.get('href'))
        similar_app_ids = [val[23:] for val in similar_app_urls if val.startswith('/store/apps/details?id=')]
        similar_or_other_app_ids = similar_or_other_app_ids + similar_app_ids 
        app_ids_searched = app_ids_searched + [app_id]*len(similar_app_ids)
        unique_app_list.at[start_index+index,'Searched Similar Apps'] = True 


    # Start similar app search 
    if(exists('./similar_app_mappings_data.xlsx')):
        app_mappings = pd.read_excel('./similar_app_mappings_data.xlsx')            
        mappings_obj = {
            "App ID": app_mappings['App ID'].to_list()+app_ids_searched,
            "Similar App ID": app_mappings['Similar App ID'].to_list()+similar_or_other_app_ids
        }
        app_mappings_df = pd.DataFrame(mappings_obj)
        app_mappings_df.to_excel (r'./similar_app_mappings_data.xlsx', index = False, header=True)
    else:
        mappings_obj = {
            "App ID": app_ids_searched,
            "Similar App ID": similar_or_other_app_ids
        }
        app_mappings_df = pd.DataFrame(mappings_obj)
        app_mappings_df.to_excel (r'./similar_app_mappings_data.xlsx', index = False, header=True)
    # Get unique URLs among similar_or_other_app_urls
    unique_similar_app_ids=[]
    seen_again = set()
    for item in similar_or_other_app_ids:
        if item not in seen_again:
            seen_again.add(item)
            unique_similar_app_ids.append(item) 
    #  Compare URLs in app_urls with unique_similar_app_urls to see which URLs are new and add them to sheet
    new_similar_apps = [id for id in unique_similar_app_ids if id not in app_ids]
    new_similar_apps_obj = {
        "App ID": unique_app_list['App ID'].to_list() + new_similar_apps,
        "Searched Similar Apps": unique_app_list['Searched Similar Apps'].to_list() + [False]*len(new_similar_apps)
    }
    updated_apps_df = pd.DataFrame(new_similar_apps_obj)
    updated_apps_df.to_excel(r'./unique_app_ids_list.xlsx', index = False, header=True)
    create_app_data_dataframe(new_similar_apps) 
    print("Total number of apps found after similar app search: ", len(app_ids+new_similar_apps))    
        
 

In [55]:
with open("config.json") as json_data_file:
    data = json.load(json_data_file)

def get_collection_or_category_urls(collection_or_category_name, collection_or_category, geographic_location):
    url = 'https://play.google.com/store/apps/' + collection_or_category +'/' + collection_or_category_name + '?gl=' + geographic_location
    http = httplib2.Http()
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    if platform.system() == 'Windows':
        driver = webdriver.Chrome(executable_path='./chromedriver.exe')
    else:
        if platform.system() == 'Darwin':
            driver = webdriver.Chrome()
    driver.implicitly_wait(30)
    # provide url(s) for driver
    driver.get(url)
    # scroll to get all apps
    scroll(driver, 2)

    # read and obtain all urls
    if platform.system() == 'Windows':
        soup_a = BeautifulSoup(driver.page_source, features='html.parser')
    else:
        if platform.system() == 'Darwin':
            soup_a = BeautifulSoup(driver.page_source, 'lxml')
    
    driver.close()

    # copy all urls
    urls = []
    for link in soup_a.find_all('a'):
        urls.append(link.get('href'))
    app_ids = [val[23:] for val in urls if val.startswith('/store/apps/details?id=')]
    # print("No. of apps found for " + collection_or_category + " " + collection_or_category_name + ":", len(app_urls))    
    # print(app_urls)
    return app_ids

if(exists('./unique_app_ids_list.xlsx')):
    unique_app_list = pd.read_excel('./unique_app_ids_list.xlsx')
    already_searched = [status for status in unique_app_list['Searched Similar Apps'].to_list() if status == True]
    if len(unique_app_list['App ID']) == len(already_searched):
        print('Start Crawling from scratch again.')
    else:
        # Search for Similar apps for 160 apps every time
        get_similar_app_ids_from_search(unique_app_list['App ID'].to_list(),len(already_searched),len(already_searched)+8, data)        
else:
    print('Starting initial Play Store Crawl for apps......')
    all_urls = []
    geo_locations = []
    for location in data['gl']:
        geo_locations.append(data['gl'][location])
    for gl_code in geo_locations:
        for config_key in data['category']:
            all_urls = all_urls + get_collection_or_category_urls(data['category'][config_key],'category',gl_code)
        for config_key in data['collection']:
            all_urls = all_urls + get_collection_or_category_urls(data['collection'][config_key],'collection',gl_code) 
    seen = set()
    initial_unique_ids = []
    for item in all_urls:
        if item not in seen:
            seen.add(item)
            initial_unique_ids.append(item)
    initial_app_obj={
        "App ID": initial_unique_ids,
        "Searched Similar Apps": [False]*len(initial_unique_ids)
    }
    initial_app_df = pd.DataFrame(initial_app_obj)
    initial_app_df.to_excel(r'./unique_app_ids_list.xlsx', index = False, header=True)
    create_app_data_dataframe(initial_unique_ids)     
    print("Total number of apps found initially: ", len(initial_unique_ids))
    get_similar_app_ids_from_search(initial_app_df['App ID'].to_list(),0,3,data) 


Starting Play Store Crawl for similar apps......


c:\Users\Raghunandhan\miniconda3\envs\canicopyyours\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if sys.path[0] == "":


Total number of apps found after similar app search:  13063


In [56]:
# remove extra characters from urls array to get only app ids
# for i in range(len(total_unique_urls)):
#     total_unique_urls[i] = total_unique_urls[i][23:]
#     print(total_unique_urls[i])
# print(len(total_unique_urls))
# print("------------------------------------------------------------------------------------------------------------")
# data_urls = get_data_safety_urls(total_unique_urls)
# print(len(data_urls))
# print(data_urls, total_unique_urls)

In [57]:
# pd.set_option('max_colwidth', 400)
# df = pd.DataFrame(data_urls, columns=['data safety urls'])
# display(df)

In [58]:
# appid_df = pd.DataFrame(app_ids, columns=['app id'])
# display(appid_df)

In [59]:
def get_app_ids_from_page(driver):
    scroll(driver, 2)

    # read and obtain all urls
    if platform.system() == 'Windows':
        soup_a = BeautifulSoup(driver.page_source, features='html.parser')
    else:
        if platform.system() == 'Darwin':
            soup_a = BeautifulSoup(driver.page_source, 'lxml')

    driver.close()

    # copy all urls
    urls = []
    for link in soup_a.find_all('a'):
        urls.append(link.get('href'))
    return [val for val in urls if val.startswith('/store/apps/details?id=')]

In [60]:
# similar_or_other_app_ids = []
# similar_app_id_mappings = []
# # app_url_list = ['/store/apps/details?id=com.whatsapp','/store/apps/details?id=com.ludo.king']

# for app_url in total_unique_urls:
#     url = data['play_store_base_url']+app_url 
#     https://play.google.com/store/search?q=com.prankcall.fakecalling.huggywuggy&c=apps
#     http = httplib2.Http()
#     chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument("--incognito")
#     if platform.system() == 'Windows':
#         driver = webdriver.Chrome(executable_path='./chromedriver.exe')
#     else:
#         if platform.system() == 'Darwin':
#             driver = webdriver.Chrome()
    
#     driver.get(url)
#     similar_app_links = driver.find_elements("xpath","//a[contains(@aria-label, 'See more information on Similar') and contains(@href, '/store/apps/collection/cluster')]")
#     # same_developer_app_links = driver.find_elements("xpath","//a[contains(@aria-label, 'See more information on') and contains(@href, '/store/apps/dev')]")
#     if len(similar_app_links)>0:
#         similar_app_links[0].click()
#         similar_app_ids = get_app_ids_from_page(driver)
#         similar_app_id_mappings.append(similar_app_ids)
#         similar_or_other_app_ids = similar_or_other_app_ids + similar_app_ids
#     else:
#         similar_app_id_mappings.append([])    

#     # urls = []
#     # for link in soup_a.find_all('a'):
#     #     urls.append(link.get('href'))
#     # app_urls = [val for val in urls if val.startswith('/store/apps/details?id=')]
# combined_app_urls = total_unique_urls + similar_or_other_app_ids
# seen_again = set()
# final_unique_urls = []
# for item in combined_app_urls:
#     if item not in seen_again:
#         seen.add(item)
#         final_unique_urls.append(item)     
# print("Total number of apps found: ", len(final_unique_urls))
# print(final_unique_urls)


        

In [61]:
# from htmldate import find_date
# date = find_date('http://gammaplay.com/privacy_policy.php')
# print(date)

In [62]:
# concatenate app id and data safety urls dfs
# df = pd.DataFrame(data_urls, columns=['data safety urls'])
# appid_df = pd.DataFrame(total_unique_urls, columns=['app id'])
# pd.set_option('max_colwidth', 400)
# all_df = pd.concat([appid_df, df], axis=1, ignore_index=True)

# all_df.columns = ['app id', 'data safety url']
# display(all_df)

In [63]:
# darr = all_df['data safety url'].to_numpy()
# print(darr)

In [64]:
# press privacy policy button on page
# if platform.system() == 'Windows':
#         driver = webdriver.Chrome(executable_path='./chromedriver.exe')
# else:
#     if platform.system() == 'Darwin':
#         driver = webdriver.Chrome()
# driver.implicitly_wait(2)
# temp = []
# df = pd.DataFrame(columns=['privacy_policy_url'])
# for i in darr:
# #     temp += press_button(driver, i)
#     df = df.append({'privacy_policy_url':press_button(driver,i)}, ignore_index=True)
# #     urls = urls.append({'privacy_policy_url': str(priv_url)}, ignore_index=True)
# #     print(i)
# # urls = url.concat(urls, df)
# # df.insert(loc=0, column = 'privacy_policy_url', value=temp)
# driver.close()
# # display(urls)

In [65]:
# display(df)

In [66]:
# display(all_df)

In [67]:
# print(len(df.index))
# print(len(all_df.index))


# merged = pd.concat([df, all_df], axis=1)
# display(merged)

In [68]:
# driver = webdriver.Chrome()
# driver.implicitly_wait(2)
# temp = press_button(driver, "https://play.google.com/store/apps/datasafety?id=com.apple.atve.androidtv.appletv")
# df.insert(loc=1, column = 'privacy policy url', value=temp)
# driver.close()

In [69]:
# link = "https://www.apple.com/legal/privacy/pdfs/apple-privacy-policy-en-ww.pdf"
# policy = []

# r = requests.get(link)
# f = io.BytesIO(r.content)

# reader = PdfFileReader(f)
# count = reader.numPages

# for i in range(count):
#     policy += reader.getPage(i).extractText().split('\n')

In [70]:
# print(policy)

In [71]:
# begin working sample of reading privacy policy html page with no dropdown menu's

# url = "https://snap.com/en-US/privacy/privacy-policy"
# url = "https://www.whatsapp.com/legal/privacy-policy"

# res = requests.get(url)
# html_page = res.content

# soup = BeautifulSoup(html_page, 'html.parser')
# text = soup.find_all(text=True)

In [72]:
# set([t.parent.name for t in text])

In [73]:
# output = ''
# blacklist = [
#     '[document]',
#    'noscript',
#     'header',
#     'html',
#     'meta',
#     'head', 
#     'input',
#     'script',
#     'link',
#     'style'
# ]

# for t in text:
#     if t.parent.name not in blacklist:
#         output += '{} '.format(t)

In [74]:
# print(output)

# end html page read